# TP2 Alejandro balladares 101118

In [1]:
!pip install pyspark
!pip install -U -q PyDrive
!apt install openjdk-8-jdk-headless -qq
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext
from pyspark.sql import SQLContext
import pandas as pd

openjdk-8-jdk-headless is already the newest version (8u292-b10-0ubuntu1~18.04).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 37 not upgraded.


In [2]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [3]:
# create the Spark Session
spark = SparkSession.builder.getOrCreate()

# create the Spark Context
sc = spark.sparkContext

1 - 34. ¿Quién es el usuario que más ha bloqueado a otros? (⭐)

2 - 19. Cantidad de contenido por planeta fuera de la tierra en la Wikipedia. (⭐)

3 - 30. Qué porcentaje de contenido geolocalizado de wikipedia NO está en la tierra (⭐)

4 - 32. ¿Cuál es el segundo contenido con más referencias geográficas asignadas? (⭐⭐)

5 - 31. Calcule la latitud y longitud promedio de los contenidos con referencias en la tierra y
diga dónde está eso (⭐⭐)

6 - 40. Si decimos que un usuario sabe un idioma cuando tiene un nivel de babel mayor o
igual a 1, para aquellos que editaron una de las versiones actuales del contenido,
¿Cuál es la tasa de revisiones sin comentario que realizan en función de los idiomas
que saben? (⭐⭐⭐)

# Punto 1 

34. ¿Quién es el usuario que más ha bloqueado a otros? (⭐)

In [4]:
id_logs ='1xbwXF1PqfUlCdPUQwdH1WDY6GrogSA3d'
downloaded = drive.CreateFile({'id': id_logs})
downloaded.GetContentFile('logs.csv')

#rdd_logs = spark.sparkContext.textFile('logs.csv')

sqlContext = SQLContext(sc)
rdd_logs = sqlContext.read.csv('logs.csv',header=True).rdd
rdd_logs.take(2)

/usr/local/lib/python3.7/dist-packages/pyspark/sql/context.py:79: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  FutureWarning


[Row(item_id='1', timestamp='2004-12-23T07:10:09Z', contributor_username='Sabbut', contributor_id='2075', contributor_ip=None, comment="el contenido era: 'Si aprendés Pensamiento Científico ... TE VA S AENTERAR'", logtype='delete', action='delete', title='Verificacionismo'),
 Row(item_id='2', timestamp='2004-12-23T07:14:09Z', contributor_username='Sabbut', contributor_id='2075', contributor_ip=None, comment="el contenido era: '{{destruir}}en algun lugar lejano pero tan cerca como el largo del cuello de una jirafa se encontraba una torre'", logtype='delete', action='delete', title='Rapunzel')]

In [5]:
rdd_logs.filter(lambda x: (x.action == 'block') or (x.action == 'reblock'))\
.map(lambda x: ((x.contributor_id, x.contributor_username if x.contributor_username != None else x.contributor_ip), 1))\
.reduceByKey(lambda x,y: x+y).reduce(lambda x,y: x if x[1] > y[1] else y)

(('34991', 'Magister Mathematicae'), 29885)

# Punto 2

19. Cantidad de contenido por planeta fuera de la tierra en la Wikipedia. (⭐)

In [6]:
id_geo_tags = '1EwtdzLu_QXCifu85wH-memXQtX5118lT'
downloaded = drive.CreateFile({'id': id_geo_tags})
downloaded.GetContentFile('geo_tags.csv')

rdd_geo_tags = sqlContext.read.csv('geo_tags.csv',header=True).rdd#.cache() nati me dijo que solo se cachea si calculo algo, no para leer
rdd_geo_tags.take(2)

[Row(gt_id='158041', gt_page_id='4328020', gt_globe='earth', gt_primary='0', gt_lat='41.50749969', gt_lon='-5.81750011', gt_dim='1000', gt_type=None, gt_name=None, gt_country=None, gt_region=None),
 Row(gt_id='5844377', gt_page_id='4670424', gt_globe='earth', gt_primary='0', gt_lat='13.73799992', gt_lon='-89.29199982', gt_dim='10000', gt_type='city', gt_name=None, gt_country=None, gt_region=None)]

In [7]:
rdd_geo_sin_Tierra = rdd_geo_tags.filter(lambda x: (x.gt_globe != 'earth') and (x.gt_globe != None)) #hay uno solo que es None pero igual lo filtro
rdd_geo_sin_Tierra.count()

11239

In [8]:
df_respuesta_2 = rdd_geo_sin_Tierra.map(lambda x: (x.gt_globe, 1)).reduceByKey(lambda x,y: (x+y))
df_respuesta_2.collect()#take(18)

[('mars', 1438),
 ('titan', 4),
 ('callisto', 3),
 ('titania', 2),
 ('hyperion', 1),
 ('tethys', 1),
 ('mimas', 1),
 ('mercury', 561),
 ('moon', 9061),
 ('ganymede', 116),
 ('venus', 24),
 ('umbriel', 1),
 ('phobos', 20),
 ('oberon', 1),
 ('io', 2),
 ('deimos', 2),
 ('enceladus', 1)]

# Punto 3

30. Qué porcentaje de contenido geolocalizado de wikipedia NO está en la tierra (⭐)

In [9]:
#rdd_geo_tags.map(lambda x: ((x.gt_page_id, x.gt_globe))).groupByKey().map(lambda x: (x[0], list(set(list(x[1]))))).filter(lambda x: (len(x[1]) > 2) and ("earth" in x[1])).take(11)
#comprobación de que hay elemenos editados fuera y dentro de la tierra

In [10]:
rdd_respuesta_3 = rdd_geo_tags.map(lambda x: (x.gt_page_id, (1 if x.gt_globe == 'earth' else 0)))
rdd_respuesta_3 = rdd_respuesta_3.reduceByKey(lambda x,y: (x + y)) # (id, n° de veces que el contenido apareció en la Tierra)
rdd_respuesta_3 = rdd_respuesta_3.map(lambda x: ((1 if x[1] == 0 else 0, 1))) #solo quiero las que no son de la Tierra
 
respuesta_3 = rdd_respuesta_3.reduce(lambda x,y: (x[0] + y[0], x[1] + y[1]))
respuesta_3

(2223, 371412)

In [11]:
(respuesta_3[0]*100)/respuesta_3[1]

0.5985267034990792

# Punto 4

32. ¿Cuál es el segundo contenido con más referencias geográficas asignadas? (⭐⭐)

In [12]:
id_contents = '1SarxhT65lTj5vztK-QnvrO625qMjCokB'

downloaded = drive.CreateFile({'id': id_contents})
downloaded.GetContentFile('contents.csv')

rdd_contents = sqlContext.read.csv('contents.csv',header=True).rdd#.cache() nati me dijo que solo se cachea si calculo algo, no para leer
rdd_contents.take(2)

[Row(title='Wikipedia:Artículos solicitados', id='5', namespace='4', revision_id='132533307', parent_revision='132516631', revision_timestamp='2021-01-19T18:40:34Z', revisor_username='Metrónomo', revisor_id='1708233', revisor_ip=None, revisor_comment='Así esta mejor'),
 Row(title='Andorra', id='7', namespace='0', revision_id='138035057', parent_revision='138027475', revision_timestamp='2021-08-31T18:27:01Z', revisor_username=None, revisor_id=None, revisor_ip='2800:CD0:FC1B:A900:C5A4:9E81:7175:2ED2', revisor_comment='Añadir contenido')]

In [13]:
rdd_geo_join_4 = rdd_geo_tags.map(lambda x: (x.gt_page_id, 1)).reduceByKey(lambda x,y: x + y )
rdd_contents_join_4 = rdd_contents.map(lambda x: (x.id, x.title))

In [14]:
rdd_respuesta_4 = rdd_geo_join_4.join(rdd_contents_join_4)
#rdd_respuesta_4.take(2)

In [15]:
rdd_respuesta_4.map(lambda x: ((x[0], x[1][1]), x[1][0])).takeOrdered(2, lambda x: -x[1])

[(('7421943', 'Wikiproyecto:Madrid/Edificios'), 1424),
 (('8329856', 'Wikiproyecto:Madrid/Artículos/Wikidata'), 1394)]

# Punto 5

31. Calcule la latitud y longitud promedio de los contenidos con referencias en la tierra y
diga dónde está eso (⭐⭐)

In [16]:
pip install geopy

In [17]:
from geopy.geocoders import Nominatim
geolocalizador = Nominatim(user_agent="my-application")

In [18]:
rdd_geo_filtrado_5 = rdd_geo_tags.filter(lambda x: (x.gt_globe == 'earth'))

In [19]:
#Promedio de todas las ubicaciones
rdd_geo_MEAN = rdd_geo_filtrado_5.map(lambda x: (float(x.gt_lat),float(x.gt_lon), 1))\
.reduce(lambda x,y: (x[0] + y[0], x[1] + y[1], x[2]+ y[2]))

respuesta_5 = (rdd_geo_MEAN[0]/rdd_geo_MEAN[2], rdd_geo_MEAN[1]/rdd_geo_MEAN[2])
respuesta_5

(32.353382400123635, -24.665538716943857)

In [20]:
ubicacion = geolocalizador.reverse(respuesta_5)
direccion = ubicacion.address
print(direccion) #es en medio del oceano atlantico norte "cerca" de Africa

#https://www.google.com/maps/place/32%C2%B021'12.2%22N+24%C2%B039'55.9%22W/@39.8831779,-28.3188261,5z/data=!4m5!3m4!1s0x0:0x0!8m2!3d32.3533824!4d-24.6655387

None


In [21]:
#ubicacion.raw

Al principio entendí que había que hacer esto que sigue, consultando me dijeron que no es lo que se pedía pero lo dejo igual como curiosidad

In [22]:
#Promedio de ubicaciones por id de contenido
rdd_geo_mean = rdd_geo_filtrado_5.map(lambda x: (x.gt_page_id, (float(x.gt_lat),float(x.gt_lon), 1)))\
.reduceByKey(lambda x,y: (x[0] + y[0], x[1] + y[1], x[2] + y[2]))\
.map(lambda x: (x[0], (x[1][0]/x[1][2], x[1][1]/x[1][2])))

In [23]:
rdd_respuesta_5 = rdd_geo_mean.map(lambda x: (x[0],(x[1][0],x[1][1], geolocalizador.reverse(str(x[1])).address))) #(id,(lat,long, lugar))
rdd_respuesta_5.take(3)

[('4670424',
  (13.600648120555554,
   -89.16110184166668,
   'Municipio de Panchimalco, Departamento de San Salvador, El Salvador')),
 ('1314296',
  (42.13186264,
   -4.23173165,
   'PP-4140, Valbuena de Pisuerga, Palencia, Castilla y León, 34465, España')),
 ('1363470',
  (11.09305573,
   -63.96805573,
   'Planta de Tratamiento de Aguas Residuales de Juan Griego, Vereda 8, Juan Griego, Urbanización Francisco Adrián, Parroquia Capital Marcano, Juan Griego, Parroquia Marcano, Municipio Marcano, Nueva Esparta, 6309, Venezuela'))]

# Punto 6

40. Si decimos que un usuario sabe un idioma cuando tiene un nivel de babel mayor o
igual a 1, para aquellos que editaron una de las versiones actuales del contenido,
¿Cuál es la tasa de revisiones sin comentario que realizan en función de los idiomas
que saben? (⭐⭐⭐)

In [24]:
id_languages = '1GC17kS3UFwcv4qoV0FkDdGH6eNgDFCR0'
downloaded = drive.CreateFile({'id': id_languages})
downloaded.GetContentFile('languages.csv')

rdd_languages = sqlContext.read.csv('languages.csv',header=True).rdd

rdd_contents.take(2)

#rdd_languages.filter(lambda x: x.babel_user == '90823').take(50)

[Row(title='Wikipedia:Artículos solicitados', id='5', namespace='4', revision_id='132533307', parent_revision='132516631', revision_timestamp='2021-01-19T18:40:34Z', revisor_username='Metrónomo', revisor_id='1708233', revisor_ip=None, revisor_comment='Así esta mejor'),
 Row(title='Andorra', id='7', namespace='0', revision_id='138035057', parent_revision='138027475', revision_timestamp='2021-08-31T18:27:01Z', revisor_username=None, revisor_id=None, revisor_ip='2800:CD0:FC1B:A900:C5A4:9E81:7175:2ED2', revisor_comment='Añadir contenido')]

In [25]:
rdd_contents_join_6 = rdd_contents.map(lambda x: (x.revisor_id, (1 if x.revisor_comment == None else 0, 1)))\
.reduceByKey(lambda x,y: (x[0] + y[0], x[1] + y[1]))

rdd_contents_join_6.take(3) #(usuario , (n° de revisiones sin comentario, revisiones totales))

[(None, (77544, 193756)), ('3615366', (238, 2905)), ('6331749', (38, 47))]

# Respuesta en formato (cantidad de idiomas, tasa)

In [26]:
rdd_languages_join = rdd_languages.filter(lambda x: (x.babel_level != '0')).map(lambda x: (x.babel_user, 1)).reduceByKey(lambda x,y: x+y)
rdd_languages_join.take(5) #(usuario, cantidad de idiomas)

[('4502458', 3), ('2050449', 7), ('4494742', 10), ('70090', 6), ('407454', 8)]

In [27]:
rdd_full_cant_idiomas = rdd_contents_join_6.leftOuterJoin(rdd_languages_join)  #(usuario ,((n° de revisiones sin comentario, revisiones totales), cant idiomas))
rdd_full_cant_idiomas.take(5)

[(None, ((77544, 193756), None)),
 ('3615366', ((238, 2905), 4)),
 ('15514', ((44, 1216), 2)),
 ('24149', ((1, 3431), None)),
 ('96461', ((956, 1955), None))]

In [28]:
rdd_final = rdd_full_cant_idiomas.map(lambda x: (x[1][1] if (x[1][1] != None) else 0, (x[1][0][0],x[1][0][1]))) #(cant idiomas,(n° de revisiones sin comentario, revisiones totales))
rdd_final.take(3)

[(0, (77544, 193756)), (4, (238, 2905)), (2, (44, 1216))]

In [29]:
rdd_respuesta_6_cant_idiomas = rdd_final.reduceByKey(lambda x,y: (x[0] + y[0], x[1] + y[1])).map(lambda x: (x[0], (x[1][0]/x[1][1])))
rdd_respuesta_6_cant_idiomas.takeOrdered(30, lambda x: -x[0])

[(34, 0.14965986394557823),
 (24, 0.050936768149882905),
 (23, 0.0),
 (22, 0.6974789915966386),
 (21, 0.5),
 (18, 0.0),
 (17, 0.14166666666666666),
 (16, 0.17582417582417584),
 (15, 0.0),
 (14, 0.2215821812596006),
 (13, 0.3333333333333333),
 (12, 0.04801720019111323),
 (11, 0.037037037037037035),
 (10, 0.09652572482438773),
 (9, 0.2247191011235955),
 (8, 0.2643062713339134),
 (7, 0.06788203086112493),
 (6, 0.1671043464279972),
 (5, 0.13079006173540886),
 (4, 0.14553954031943903),
 (3, 0.12955704377300453),
 (2, 0.1930673968510797),
 (1, 0.2489639885637187),
 (0, 0.10450331009720965)]

# Formato (Idioma, tasa) lo dejo porque ya lo tenía hecho

In [30]:
rdd_languages_join_2 = rdd_languages.filter(lambda x: (x.babel_level != '0')).map(lambda x: ((x.babel_user, x.babel_lang)))

rdd_languages_join_2.take(2)

[('4502458', 'FR'), ('2050449', 'ab')]

In [31]:
rdd_full_idiomas = rdd_contents_join_6.leftOuterJoin(rdd_languages_join_2)
rdd_full_idiomas.take(2)

[(None, ((77544, 193756), None)), ('3615366', ((238, 2905), 'ca'))]

In [32]:
rdd_respuesta_6_idiomas = rdd_full_idiomas.map(lambda x: (x[1][1] if x[1][1] != None else 'ninguno', (x[1][0][0], x[1][0][1])))\
.reduceByKey(lambda x,y: (x[0] + y[0], x[1] + y[1]))\
.map(lambda x: (x[0],((x[1][0]/x[1][1]),x[1][1]))).filter(lambda x: x[1][1] > 1000)

In [33]:
rdd_respuesta_6_idiomas.take(2)

[('ninguno', (0.10450331009720965, 3308211)),
 ('gl', (0.15099750049836688, 65213))]

In [34]:
#rdd_respuesta_6_idiomas.collect()#.take(191)
rdd_respuesta_6_idiomas.takeOrdered(10, lambda x: -x[1][0])#and x[1][1] < 1000)

[('ext', (0.33361250697934114, 7164)),
 ('gn', (0.30995762711864405, 4720)),
 ('qu', (0.23919233756148073, 3863)),
 ('lad', (0.23729384045775834, 2971)),
 ('nah', (0.23601249644987218, 3521)),
 ('ie', (0.2229470452801228, 2606)),
 ('eu', (0.2038064554620846, 16761)),
 ('bg', (0.20163636363636364, 5500)),
 ('ia', (0.19873921698739216, 3014)),
 ('pl', (0.19477621735135844, 7619))]